In [1]:
from dotenv import load_dotenv
load_dotenv("dev.env")
import os

db_name = os.getenv("db_name")
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
website = os.getenv("website")
city_link = os.getenv("city_link")
chrome_path = os.getenv("chrome_path")
selenium_chrome_driver_path = os.getenv("selenium_chrome_driver_path")

In [2]:
import psycopg2
import time
import math
import numpy as np
import os.path
from datetime import date, timedelta, datetime

conn_string = 'host={pghost} port={pgport} dbname={pgdatabase} user={pguser} password={pgpassword}'.format(pgdatabase=db_name,pguser=db_username,pgpassword=db_password,pghost=db_host,pgport=db_port)
conn=psycopg2.connect(conn_string)
cur=conn.cursor()

def check_if_table_exists(schema,table):
    cur.execute("select exists(select * from information_schema.tables where table_schema='{schema}' AND table_name='{table}')".format(schema=schema, table=table))
    return cur.fetchone()[0]

def check_if_index_exists(index):
    cur.execute("SELECT EXISTS(SELECT * FROM PG_CLASS WHERE relname = '{index}')".format(index=index))
    return cur.fetchone()[0]

def check_if_file_exists(filename):
    return os.path.isfile(filename)

def execute_mogrify(conn, df, schema, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = '"'+'","'.join(list(df.columns))+'"'
    cursor = conn.cursor()    
    try:
        for tup in tuples:
            query  = """INSERT INTO "{schema}"."{table}"({cols}) VALUES ({values}) ON CONFLICT DO NOTHING""".format(schema=schema,table=table, cols=cols, values=",".join(map(str,tup)))
            cursor.execute(query)
            conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

def df_column_conversation(df, column_name, type):
    if(type == 'timestamp'):
        df[column_name] = df[column_name].apply(lambda x: f"'{x}'::timestamp")
    if(type == 'text'):
        df[column_name] = df[column_name].str.replace("'","").apply(lambda x: f"'{x}'")
    if(type == 'date'):
        df[column_name] = df[column_name].apply(lambda x: f"'{x}'::date")
    if(type == 'numeric'):
        df[column_name] = df[column_name].str.replace(',','.')
    if(type == 'integer'):
        df[column_name] = df[column_name].str.replace(',','.').apply(float).apply(int)

In [3]:
if(check_if_table_exists('ODS','EXT_FB_RESTAURANT')):
    print('Table ODS.EXT_FB_RESTAURANT already exists.')   
else:
    start_time = math.trunc(time.time())
    cur.execute("""
    CREATE TABLE "ODS"."EXT_FB_RESTAURANT"
    (
    "RESTAURANT_ID" text NOT NULL,
    "RESTAURANT_NAME" text,
    "RESTAURANT_LINK" text,    
    CONSTRAINT "RESTAURANT_ID" UNIQUE ("RESTAURANT_ID")
    );
    """)
    cur.execute('COMMIT;')
    end_time = math.trunc(time.time())
    print("Table ODS.EXT_FB_RESTAURANT created in {execute_time} seconds.".format(execute_time=end_time-start_time))

if(check_if_table_exists('ODS','EXT_FB_MENU')):
    print('Table ODS.EXT_FB_MENU already exists.')   
else:
    start_time = math.trunc(time.time())
    cur.execute("""
    CREATE TABLE "ODS"."EXT_FB_MENU"
    (
    "PRODUCT_ID" text NOT NULL,
    "RESTAURANT_ID" text,
    "CATEGORY_NAME" text,
    "PRODUCT_NAME" text,
    "PRODUCT_DESCRIPTION" text,
    "PRODUCT_LISTED_PRICE" text,
    "PRODUCT_PRICE" text,
    "DISCOUNT" boolean,
    "DESIGN_TYPE" text,
    "DATE" date,
    CONSTRAINT "PRODUCT_ID" UNIQUE ("PRODUCT_ID")
    );
    """)
    cur.execute('COMMIT;')
    end_time = math.trunc(time.time())
    print("Table ODS.EXT_FB_MENU created in {execute_time} seconds.".format(execute_time=end_time-start_time))

Table ODS.EXT_FB_RESTAURANT already exists.
Table ODS.EXT_FB_MENU already exists.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

options = Options()
options.binary_location = chrome_path

restaurant_list = []

driver = webdriver.Chrome(options=options, executable_path=selenium_chrome_driver_path)
driver.get(city_link)
time.sleep(5)
for i in range(25):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
city_restaurant_groups = driver.find_elements_by_class_name("restaurant-main-info")
for restaurant in city_restaurant_groups:
    restaurant_name = restaurant.find_element_by_class_name("restaurant-display-name").text
    restaurant_name = restaurant_name.replace("YENİ ", "")
    restaurant_link = restaurant.find_element_by_class_name("restaurant-display-name").find_element_by_xpath(".//a").get_attribute('href')
    restaurant_id = restaurant_link.split("/")[-1]
    if(len(restaurant_link) < 2):
        continue
    restaurant_list.append([restaurant_id,restaurant_name,restaurant_link])
restaurant_df = pd.DataFrame(restaurant_list, columns=["RESTAURANT_ID","RESTAURANT_NAME","RESTAURANT_LINK"])

df_column_conversation(restaurant_df, 'RESTAURANT_ID', 'text')
df_column_conversation(restaurant_df, 'RESTAURANT_NAME', 'text')
df_column_conversation(restaurant_df, 'RESTAURANT_LINK', 'text')
execute_mogrify(conn,restaurant_df,"ODS","EXT_FB_RESTAURANT")
restaurant_df

In [5]:
sql_command = """
    SELECT
    "RESTAURANT_ID"
    FROM "ODS"."EXT_FB_RESTAURANT" EFR
    WHERE 1=1
    AND NOT EXISTS(SELECT NULL FROM "ODS"."EXT_FB_MENU" EFM WHERE EFR."RESTAURANT_ID" = EFM."RESTAURANT_ID");
    """
restaurant_df = pd.read_sql(sql_command,conn)

In [6]:
for i in range(len(restaurant_df)):
    sublink = restaurant_df.loc[i,"RESTAURANT_ID"]
    restaurant_link = "{website}/{sublink}".format(website=website,sublink=sublink)
    
    driver.get(restaurant_link)
    time.sleep(5)

    try:
        if("sipariş verebilirsiniz." in driver.find_element_by_xpath('//*[@id="restaurantDetail"]/div/div[2]/h3').text):
            continue
        else:
            pass
    except Exception:
        pass
    
    menu = driver.find_element_by_xpath('//*[@id="restaurant_menu"]')
    categories = menu.find_elements_by_xpath('//*[contains(@id,"menu_")]')

    menu_list = []

    for category in categories:
        category_name = category.find_element_by_xpath(".//b").text
        for product in category.find_elements_by_xpath(".//div[2]/ul/li"):
            try:
                design_type = "list"
                try:
                    product_id = product.find_elements_by_class_name("getProductDetail")[-1].get_attribute('data-product-id')
                    product_name = product.find_elements_by_class_name("getProductDetail")[-1].text
                except:
                    product_id = product.find_element_by_xpath(".//strong").get_attribute('data-product-id')
                    product_name = product.find_element_by_xpath(".//strong").text
                    design_type = "card"
                try:
                    product_description = product.find_element_by_class_name("product-desc").text
                    product_price = product.find_element_by_class_name("price").text    
                except:
                    product_description = product.find_element_by_class_name("productInfo").text
                    product_price = product.find_element_by_class_name("newPrice").text
                    if(not(design_type=="card")):
                        design_type = "box"
                discount = "TRUE"
                try:              
                    if(design_type=="list"):
                        product_listed_price = product.find_element_by_class_name("listed-price").text
                    if(design_type in ["card","box"]):
                        product_listed_price = product.find_element_by_class_name("listedPrice").text
                except:
                    product_listed_price = product_price
                    discount = "FALSE"
                menu_list.append([product_id,sublink,category_name,product_name,product_description,product_listed_price,product_price,discount,design_type])
            except:
                continue
    menu_df = pd.DataFrame(menu_list, columns=["PRODUCT_ID","RESTAURANT_ID","CATEGORY_NAME","PRODUCT_NAME","PRODUCT_DESCRIPTION","PRODUCT_LISTED_PRICE","PRODUCT_PRICE","DISCOUNT","DESIGN_TYPE"])    
    menu_df = menu_df[menu_df['PRODUCT_ID'].str.len() > 0]
    menu_df = menu_df[menu_df['PRODUCT_NAME'].str.len() > 0]

    df_column_conversation(menu_df, 'PRODUCT_ID', 'text')
    df_column_conversation(menu_df, 'RESTAURANT_ID', 'text')
    df_column_conversation(menu_df, 'CATEGORY_NAME', 'text')
    df_column_conversation(menu_df, 'PRODUCT_NAME', 'text')
    df_column_conversation(menu_df, 'PRODUCT_DESCRIPTION', 'text')
    df_column_conversation(menu_df, 'PRODUCT_LISTED_PRICE', 'text')
    df_column_conversation(menu_df, 'PRODUCT_PRICE', 'text')
    df_column_conversation(menu_df, 'DESIGN_TYPE', 'text')    
    menu_df['DATE'] = "'"+ datetime.strftime(date.today(), "%Y-%m-%d") + "'::date"
    execute_mogrify(conn,menu_df,"ODS","EXT_FB_MENU")
menu_df

In [7]:
sql_command = """
    SELECT
    "RESTAURANT_ID"
    FROM "ODS"."EXT_FB_RESTAURANT" EFR
    WHERE 1=1
    AND NOT EXISTS(SELECT NULL FROM "ODS"."EXT_FB_COMMENT" EFC WHERE EFR."RESTAURANT_ID" = EFC."RESTAURANT_ID");
    """
restaurant_df = pd.read_sql(sql_command,conn)
restaurant_df

,RESTAURANT_ID
0,ghost-street-food-kadikoy-caddebostan-mah-ista...
1,firinda-pizza-atasehir-esatpasa-mah-istanbul
2,mancinis-pizza-bakirkoy-kartaltepe-mah-incirli...
3,beppe-pizzeria-kadikoy-acibadem-mah-istanbul
4,lokanta-fornello-kadikoy-rasimpasa-mah-istanbul
...,...
669,nak-restaurant-patisserie-esenler-oruc-reis-ma...
670,peggys-pizza-besiktas-arnavutkoy-mah-istanbul
671,little-caesars-pizza-pendik-velibaba-mah-istanbul
672,sbarro-pizza-atasehir-kucukbakkalkoy-mah-istanbul


In [8]:
for i in range(len(restaurant_df)):
    sublink = restaurant_df.loc[i,"RESTAURANT_ID"]
    last_comment_page_url = "{website}/{sublink}?section=comments&page=9999".format(website=website,sublink=sublink)
    
    driver.get(last_comment_page_url)
    time.sleep(0.1)

    comments_list = []

    last_comment_page_redirect_url = driver.current_url
    last_comment_page_number = int(last_comment_page_redirect_url.replace("&status=closed","").replace("{website}/{sublink}?section=comments&page=".format(website=website,sublink=sublink),""))

    for page_number in range(1, last_comment_page_number+1):
        current_comment_page_url = "{website}/{sublink}?section=comments&page={page_number}".format(website=website,sublink=sublink,page_number=page_number)
        driver.get(current_comment_page_url)
        time.sleep(1)

        try:
            if("sipariş verebilirsiniz." in driver.find_element_by_xpath('//*[@id="restaurantDetail"]/div/div[2]/h3').text):
                continue
            else:
                pass
        except Exception:
            pass

        try:
            driver.find_element(By.XPATH, '//*[@id="alternative-restaurant-popup"]/div[1]/div[2]/img').click(); #Closing pop-up
        except Exception:
            pass   

        #driver.find_element(By.XPATH, '//*[@id="restaurantDetail"]/div[2]/div[1]/ul/li[4]/a').click(); #Clicking comments

        comment_list = driver.find_element_by_xpath('//*[@id="restaurant_comments"]/div[4]').find_elements_by_class_name("comments-body")
    
        for comment in comment_list:
            try:
                username = comment.find_element_by_class_name("userName").text
                comment_text = comment.find_element_by_xpath('.//p').text
                comment_date = comment.find_element_by_class_name("commentDate").text
                speed = comment.find_element_by_class_name("speed").text
                serving = comment.find_element_by_class_name("serving").text
                flavour = comment.find_element_by_class_name("flavour").text 
                comments_list.append([sublink, username, comment_text, comment_date, speed, serving, flavour])
            except Exception:
                pass
    comment_df = pd.DataFrame(comments_list, columns=["RESTAURANT_ID","USERNAME","COMMENT_TEXT","COMMENT_DATE","SPEED","SERVING","FLAVOUR"])
    df_column_conversation(comment_df, 'RESTAURANT_ID', 'text')
    df_column_conversation(comment_df, 'USERNAME', 'text')
    df_column_conversation(comment_df, 'COMMENT_TEXT', 'text')
    df_column_conversation(comment_df, 'COMMENT_DATE', 'text')
    df_column_conversation(comment_df, 'SPEED', 'text')
    df_column_conversation(comment_df, 'SERVING', 'text')
    df_column_conversation(comment_df, 'FLAVOUR', 'text')
    comment_df['DATE'] = "'"+ datetime.strftime(date.today(), "%Y-%m-%d") + "'::date"
    execute_mogrify(conn,comment_df,"ODS","EXT_FB_COMMENT")

In [21]:
comment_df

,RESTAURANT_ID,USERNAME,COMMENT_TEXT,COMMENT_DATE,SPEED,SERVING,FLAVOUR
0,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...z,Peperoni favorim.,3 gün önce,Hız: 10,Servis: 10,Lezzet: 10
1,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...d,Daha kötüsünü dominosda yemedim.,3 gün önce,Hız: 9,Servis: 9,Lezzet: 1
2,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...r,İnanılmaz yedigim en iyi pizzalardan biri ola ...,3 gün önce,Hız: 10,Servis: 10,Lezzet: 10
3,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...l,"Biraz yağlı ve hamuru da aşırı inceydi, bunlar...",4 gün önce,Hız: 10,Servis: 8,Lezzet: 9
4,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...t,Cok lezzetli fakat fiyat cok yuksek...,4 gün önce,Hız: 10,Servis: 10,Lezzet: 10
...,...,...,...,...,...,...,...
129,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...4,Gerçektende en iyisi sizsiniz.,2 ay önce,Hız: 10,Servis: 10,Lezzet: 10
130,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...e,Çok teşekkür ederimçok hızlı geldi öncelikle v...,2 ay önce,Hız: 10,Servis: 10,Lezzet: 10
131,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...9,Hamurunuzun çıtırlığına bayıldımm kenarları bi...,2 ay önce,Hız: 10,Servis: 10,Lezzet: 10
132,pizza-dante-sisli-ergenekon-mah-pangalti-istanbul,...l,Bundan yaklasik 1 sene once milanoda bulunmust...,2 ay önce,Hız: 10,Servis: 10,Lezzet: 10
